In [ ]:
import pandas as pd
import bs4
import requests

In [ ]:
API_KEY = 'YOUR_API_KEY'
REGION_CODES = ['%02d' % num for num in range(1,17)]

In [ ]:
df = pd.DataFrame()

for REGION_CODE in REGION_CODES:
    
    PAGE = 1
    
    while True:
        
        # API 호출
        API = f'http://openapi.forest.go.kr/openapi/service/trailInfoService/getforeststoryservice?mntnInfoAraCd={REGION_CODE}&pageNo={PAGE}&serviceKey={API_KEY}'
        response = requests.get(API).text
        xmlobj = bs4.BeautifulSoup(response, 'lxml-xml')
        
        # ITEM 별로 파싱
        rows = xmlobj.findAll('item')
        
        if len(rows) == 0:
            print(f'== {PAGE}페이지에는 데이터가 존재하지 않습니다. 지역을 변경합니다. ==')
            break
        
        # 페이지별로 ITEM 수집
        rowList = []
        column_nameList = []
        valueList = []
        
        # Value 크롤링
        for i in range(0, len(rows)):
            columns = rows[i].find_all()

            for j in range(0, len(columns)):
                
                # 처음에 Column명 지정을 위함!
                if i == 0:
                    column_nameList.append(columns[j].name)
                
                eachColumn = columns[j].text
                valueList.append(eachColumn)
                
            rowList.append(valueList)
            valueList = []    
        
        # 데이터프레임에 추가
        tmp = pd.DataFrame(rowList, columns=column_nameList)
        df = df.append(tmp, ignore_index=True)
        
        print(f'== {REGION_CODE}지역 {PAGE}페이지 완료 ==')
        PAGE += 1
        
print('========== 크롤링 완료!! ==========')

In [ ]:
df = df.drop_duplicates().reset_index(drop=True)
df

In [ ]:
df.to_excel('../output/산정보_API(산림청_산 정보 조회).xlsx', index=False)